In [1]:
import pandas as pd
from datetime import date, timedelta

In [2]:
file_path = "https://www.dhhs.vic.gov.au/ncov-covid-cases-by-age-group-csv"
df = pd.read_csv(file_path)

In [3]:
df.tail()

,diagnosis_date,agegroup
740246,2022-03-29,40-49
740247,2022-03-29,60-69
740248,2022-03-29,0-9
740249,2022-03-29,30-39
740250,2022-03-29,50-59


In [4]:
df_pivot = pd.pivot_table(df, index = 'diagnosis_date', columns = 'agegroup', 
                         values = 'agegroup', aggfunc = len, fill_value = 0)

In [5]:
df_pivot.tail()

agegroup,0-9,10-19_,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+
diagnosis_date,,,,,,,,,,
2022-03-25,276,416,529,543,465,374,242,142,76,17
2022-03-26,282,362,386,473,410,300,229,143,62,13
2022-03-27,288,355,350,410,406,299,206,137,63,14
2022-03-28,307,414,412,463,467,342,236,130,71,19
2022-03-29,388,567,581,679,585,454,318,181,61,13


In [6]:
# check that the latest date's sum matches the number of cases reported by Vic govt
# the latest row's date is always yesterday

yesterday = date.today() - timedelta(days = 1)

# the dates in the pivot table are strings
# so use isoformat() to convert date into string before looking it up in the table
df_pivot.loc[yesterday.isoformat()].sum()

KeyError: '2022-03-30'

In [ ]:
# take only the 0-9 and 10-19 cases, from 5 Dec onwards
# for each date, store the previous date's cases; this matches what I've done for NSW
# eg. on 8 Dec, the Vic cases are those reported on the 7th

first = date(2021, 12, 5)
today = date.today()

df_0_19 = pd.DataFrame(columns = ['date', 'cases_0_9', 'cases_10_19'])

curr = first
prev = first - timedelta(days = 1)

while (curr < today):
    
        df_0_19 = df_0_19.append({'date': curr.isoformat(), 'cases_0_9': df_pivot.loc[prev.isoformat(), '0-9'], 
                                  'cases_10_19': df_pivot.loc[prev.isoformat(), '10-19_']}, ignore_index = True)
        prev = curr
        curr = curr + timedelta(days = 1)

In [ ]:
df_0_19.tail()

In [ ]:
df_0_19.to_csv("../data/cases_0_19.csv")